# 1. Combinar archivos 
#### Archivo de enrgia extraida con archivo de empresas

In [1]:
import pandas as pd
import glob
import re
import os
import numpy as np

# Configuración
EMPRESAS_FILE = './data/1_empresas.xlsx'
INPUT_PATTERN = './downloads/extracted_precios_c_ret_*.xlsx'
OUTPUT_PREFIX = './downloads/precios_empresas_'

def procesar_archivos():
    # Cargar el mapeo de empresas
    try:
        df_empresas = pd.read_excel(EMPRESAS_FILE)
        
        if 'AGENTE' not in df_empresas.columns or 'EMPRESA' not in df_empresas.columns:
            print("Error: El archivo empresas.xlsx debe contener columnas 'AGENTE' y 'EMPRESA'")
            return
            
        df_empresas['AGENTE'] = df_empresas['AGENTE'].astype(str).str.strip()
        mapeo_empresas = dict(zip(df_empresas['AGENTE'], df_empresas['EMPRESA']))
        nombres_empresas = {k: k for k in df_empresas['AGENTE'].unique()}
        
    except Exception as e:
        print(f"Error cargando {EMPRESAS_FILE}: {str(e)}")
        return

    # Procesar cada archivo de entrada
    for input_file in glob.glob(INPUT_PATTERN):
        file_number = re.search(r'extracted_precios_c_ret_(\d+)\.xlsx', input_file)
        if not file_number:
            print(f"Formato de nombre inválido: {input_file}")
            continue
        
        file_number = file_number.group(1)
        output_file = f"{OUTPUT_PREFIX}{file_number}.xlsx"
        
        if os.path.exists(output_file):
            print(f"Archivo {output_file} ya existe. Omitiendo.")
            continue

        try:
            # --- LECTURA ADAPTATIVA DEL ARCHIVO ---
            # Encontrar fila que contiene 'AGENTE'
            df_temp = pd.read_excel(input_file, header=None)
            header_idx = None
            for idx, row in df_temp.iterrows():
                if not row.empty and any(
                    re.search(r'\bAGENTE\b', str(cell), re.IGNORECASE) 
                    for cell in row if pd.notna(cell)
                ):
                    header_idx = idx
                    break

            if header_idx is None:
                print(f"Error: No se encontró 'AGENTE' en {input_file}")
                continue

            # Leer datos ignorando filas basura
            skip_rows = list(range(0, header_idx))
            df_energia = pd.read_excel(input_file, skiprows=skip_rows)
            
            # Limpiar nombres de columnas
            df_energia.columns = [
                col.strip() if isinstance(col, str) else col 
                for col in df_energia.columns
            ]

            # Identificar columna AGENTE
            agent_col = next(
                (col for col in df_energia.columns 
                 if 'agente' in str(col).lower()), 
                None
            )
            
            if not agent_col:
                print(f"Error: Columna 'AGENTE' no encontrada en {input_file}")
                continue
                
            if agent_col != 'AGENTE':
                df_energia = df_energia.rename(columns={agent_col: 'AGENTE'})
            
            # --- PREPROCESAMIENTO DE DATOS ---
            # Convertir y limpiar AGENTE
            df_energia['AGENTE'] = (
                df_energia['AGENTE']
                .astype(str)
                .str.strip()
                .replace({'nan': np.nan, '': np.nan})
            )
            
            # Eliminar filas sin agente
            df_energia = df_energia.dropna(subset=['AGENTE'])
            
            # Normalizar nombres
            def normalizar_nombre(x):
                x = str(x).strip()
                # Coincidencia exacta
                if x in nombres_empresas:
                    return nombres_empresas[x]
                # Búsqueda flexible (sin espacios/guiones)
                x_clean = re.sub(r'[-\s]', '', x).lower()
                for k in nombres_empresas:
                    k_clean = re.sub(r'[-\s]', '', k).lower()
                    if k_clean == x_clean:
                        return nombres_empresas[k]
                return x
            
            df_energia['AGENTE_NORMALIZADO'] = df_energia['AGENTE'].apply(normalizar_nombre)
            df_energia['EMPRESA'] = df_energia['AGENTE_NORMALIZADO'].map(mapeo_empresas)
            
            # Reordenar columnas
            cols = ['AGENTE_NORMALIZADO', 'EMPRESA'] + [
                col for col in df_energia.columns 
                if col not in ['AGENTE_NORMALIZADO', 'EMPRESA', 'AGENTE']
            ]
            
            # Guardar resultado
            df_resultado = df_energia[cols].rename(columns={'AGENTE_NORMALIZADO': 'AGENTE'})
            df_resultado.to_excel(output_file, index=False)
            print(f"Procesado: {input_file} -> {output_file}")

            # Reportar agentes no mapeados
            if df_resultado['EMPRESA'].isna().any():
                missing = df_resultado[df_resultado['EMPRESA'].isna()]['AGENTE'].unique()
                print(f"  Advertencia: {len(missing)} agentes sin empresa asignada")
                print(f"  Ejemplos: {list(missing)[:5]}")

        except Exception as e:
            # CORRECCIÓN: Error de sintaxis en la línea siguiente
            print(f"Error procesando {input_file}: {str(e)}")  # Se eliminó el paréntesis extra
            error_file = f"{OUTPUT_PREFIX}ERROR_{file_number}.xlsx"
            pd.read_excel(input_file).to_excel(error_file, index=False)
            print(f"  Copia de error guardada en: {error_file}")

if __name__ == "__main__":
    print("Iniciando procesamiento...")
    procesar_archivos()
    print("Proceso completado.")

Iniciando procesamiento...
Procesado: ./downloads\extracted_precios_c_ret_0123.xlsx -> ./downloads/precios_empresas_0123.xlsx
  Advertencia: 7 agentes sin empresa asignada
  Ejemplos: ['CARGOS POR RETIROS DE ENERGIA Y POTENCIA (sin IVA)', '2023-01-01 00:00:00', 'AGENTE', 'San Buenaventura', 'Uyuni - Uyuni']
Procesado: ./downloads\extracted_precios_c_ret_0124.xlsx -> ./downloads/precios_empresas_0124.xlsx
  Advertencia: 15 agentes sin empresa asignada
  Ejemplos: ['CARGOS POR RETIROS DE ENERGIA Y POTENCIA (sin IVA)', 'enero de 2024', 'AGENTE', 'Brechas', 'Troncos']
Procesado: ./downloads\extracted_precios_c_ret_0125.xlsx -> ./downloads/precios_empresas_0125.xlsx
  Advertencia: 12 agentes sin empresa asignada
  Ejemplos: ['CARGOS POR RETIROS DE ENERGÍA Y POTENCIA (sin IVA)', 'enero de 2025', 'AGENTE', 'Urubó 115 kV', 'Brechas']
Procesado: ./downloads\extracted_precios_c_ret_0223.xlsx -> ./downloads/precios_empresas_0223.xlsx
  Advertencia: 7 agentes sin empresa asignada
  Ejemplos: ['CAR

# 2. Merge archivos energia

In [2]:
import pandas as pd
import glob
from datetime import datetime

def consolidar_en_formato_largo():
    archivos = sorted(glob.glob('./downloads/precios_empresas_*.xlsx'))
    if not archivos:
        print("No se encontraron archivos.")
        return

    registros = []

    for archivo in archivos:
        try:
            periodo = archivo.split('_')[-1].split('.')[0]
            mes = int(periodo[:2])
            año = 2000 + int(periodo[2:])
            fecha = datetime(año, mes, 1)

            df = pd.read_excel(archivo)
            if 'AGENTE' not in df.columns:
                print(f"Omitido: {archivo} no tiene columna AGENTE.")
                continue

            for col in df.columns:
                if col in ['AGENTE', 'EMPRESA']:
                    continue
                temp = df[['AGENTE', 'EMPRESA', col]].copy()
                temp['FECHA'] = fecha
                temp['VARIABLE'] = col
                temp = temp.rename(columns={col: 'VALOR'})
                registros.append(temp)

        except Exception as e:
            print(f"Error procesando {archivo}: {e}")
            continue

    if not registros:
        print("No se pudo consolidar ningún archivo válido.")
        return

    df_largo = pd.concat(registros, ignore_index=True)
    df_largo = df_largo[['FECHA', 'AGENTE', 'EMPRESA', 'VARIABLE', 'VALOR']]

    df_largo.to_excel("./downloads/serie_temporal_precios.xlsx", index=False)
    print("Consolidación completada en formato largo.")
    print(f"Filas totales: {len(df_largo)}")
    print(f"Archivo guardado como 'serie_temporal_precios.xlsx'")

if __name__ == "__main__":
    consolidar_en_formato_largo()

Consolidación completada en formato largo.
Filas totales: 5378
Archivo guardado como 'serie_temporal_precios.xlsx'


# 3. Creacion de serie de tiempo de energia

In [3]:
import pandas as pd

# Leer archivo
df = pd.read_excel("./downloads/serie_temporal_precios.xlsx")

# Convertir FECHA y eliminar filas sin fecha válida
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce', unit='d')
df = df.dropna(subset=['FECHA'])

# Limpiar espacios en columnas clave
df['AGENTE'] = df['AGENTE'].astype(str).str.strip()
df['EMPRESA'] = df['EMPRESA'].astype(str).str.strip()
df['VARIABLE'] = df['VARIABLE'].astype(str).str.strip()

# Filtrar sólo las variables de interés exactas (con mayúsculas y espacios)
df = df[df['VARIABLE'].isin(['Precio Energía USD/MWh', 'Precio Potencia USD/kW'])]

# Asegurar que VALOR sea numérico
df['VALOR'] = pd.to_numeric(df['VALOR'], errors='coerce')
df = df.dropna(subset=['VALOR'])

# Crear MES_ANIO
df['MES_ANIO'] = df['FECHA'].dt.strftime('%m%Y')

# Reemplazar valores faltantes en AGENTE y EMPRESA con forward fill
df['AGENTE'] = df['AGENTE'].replace('nan', pd.NA).fillna(method='ffill')
df['EMPRESA'] = df['EMPRESA'].replace('nan', pd.NA).fillna(method='ffill')

# Crear columna pivote combinando variable y mes_año (con espacios y mayúsculas)
df['COLUMNA'] = df['VARIABLE'] + ' ' + df['MES_ANIO']

# Pivotear el DataFrame
df_pivot = df.pivot_table(index=['AGENTE', 'EMPRESA'], columns='COLUMNA', values='VALOR', aggfunc='first')

# Función para extraer año y mes para ordenar correctamente
def extraer_anio_mes(col_name):
    fecha = col_name.split()[-1]  # '012023'
    mes = int(fecha[:2])
    anio = int(fecha[2:])
    return (anio, mes)

# Obtener lista de columnas
cols = df_pivot.columns.tolist()

# Separar en Energía y Potencia
energia_cols = sorted([c for c in cols if c.startswith('Precio Energía USD/MWh')], key=extraer_anio_mes)
potencia_cols = sorted([c for c in cols if c.startswith('Precio Potencia USD/kW')], key=extraer_anio_mes)

# Concatenar para orden final
cols_ordenadas = energia_cols + potencia_cols

# Reordenar columnas
df_pivot = df_pivot[cols_ordenadas]

# Resetear índice
df_final = df_pivot.reset_index()

# Guardar resultado
df_final.to_excel("./downloads/serie_precios_cronologica.xlsx", index=False)


C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_26708\938526937.py:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['AGENTE'] = df['AGENTE'].replace('nan', pd.NA).fillna(method='ffill')
C:\Users\daniel.canedo\AppData\Local\Temp\ipykernel_26708\938526937.py:27: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['EMPRESA'] = df['EMPRESA'].replace('nan', pd.NA).fillna(method='ffill')


# 4. Depuracion de serie de tiempo de energia

In [4]:
import pandas as pd

# Leer el archivo Excel
df = pd.read_excel("./downloads/serie_precios_cronologica.xlsx")

# Lista de agentes a eliminar (como lista plana)
agentes_a_eliminar = [
    "TOTAL - CESSA", "TOTAL - CRE R.L.", "TOTAL CRE", "TOTAL - DELAPAZ", 
    "TOTAL - ELFEC", "TOTAL - ENDE", "TOTAL ENDE DELBENI S.A.M.",
    "TOTAL - ENDE DEORURO S.A.", "TOTALES", "Tipo de cambio",
    "TOTAL - SEPSA", "TOTAL - SETAR"
]

# Filtrar y eliminar las filas no deseadas
df = df[~df["AGENTE"].isin(agentes_a_eliminar)]

# Verificar agentes únicos después del filtro
print(df["AGENTE"].unique())

# Guardar el DataFrame resultante
df.to_excel("./data/serie_precios.xlsx", index=False)

['Arboleda' 'Arocagua' 'Bermejo' 'Brechas' 'Brechas 115' 'Brechas 69'
 'Bélgica' 'CERAMICA GUADALQUIVIR' 'CM  Karachipampa' 'COBOCE'
 'Camiri - Cordillera' 'Caranavi' 'Carrasco' 'Catavi' 'Catavi 115'
 'Chilcobija' 'Chimore' 'Chimoré' 'Choquetanga' 'Chuspipata'
 'Contorno Bajo' 'Cumbre' 'Don Diego' 'ECEBOL Potosí' 'EMDEECRUZ'
 'EMPACAR S.A.' 'EMVINTO - COMIBOL' 'Guaracachi' 'Guarayos' 'Irpa Irpa'
 'Jeruyo' 'Kenko' 'LAS LOMAS' 'La Plata' 'Las Carreras' 'Litio - Lipez'
 'Litio 115 kV' 'Lucianita' 'MINERA SAN CRISTOBAL S.A.' 'Mariaca' 'Ocuri'
 'Pagador' 'Palca' 'Paracaya' 'Paraíso' 'Portugalete' 'Potosi 115'
 'Potosi 69' 'Potosí' 'Potosí 115' 'Punutuma' 'Qollpana' 'Sacaca'
 'San Borja' 'San Buenaventura' 'San Ignacio de Moxos' 'San José'
 'San Julián' 'Santivañez' 'Sucre' 'Sucre - Aranjuez' 'Sucre - Fancesa'
 'Sucre 115' 'Tarija' 'Tazna' 'Telamayu' 'Torre Huayco' 'Trinidad'
 'Troncos' 'Troncos - Las Misiones' 'Troncos 115'
 'Troncos 115 (Las Misiones)' 'Urubó' 'Urubó 115' 'Urubó 115 kV' 'U